# Construindo um dataset para Phenotype Prediction usando openSNP

## Contexto

Existe um campo de estudo chamado DNA Forensics ou Forensic DNA Phenotyping, que pode se utilizar da predição de fenótipos baseado em coleta de material genético.

A predição de características externamente visíveis pode ajudar a polícia a estreitar os caminhos de um investigação e ajudar a solucionar um crime mais rápido.

Com material genético em mãos é possível buscar mutações dos genes chamadas SNPs (Single-Nucleotide Polymorphisms), algumas destas mutações podem estão associadas a fenótipos, como cor dos olhos, cor da pele, cor do cabelo e etc...

Para auxiliar nesta tarefa de predizer fenótipos baseado no material genético, podem ser aplicados modelos de Aprendizado de Máquina. Para isso, são necessários... DADOS!!! E é sobre a coleta e tratamento destes dados que se trata este trabalho. 

## Introdução

A plataforma [openSNP](https://opensnp.org/) permite que clientes de testes genéticos publiquem seus resultados e compartilhem seus fenótipos.

Usando esse plataforma foi possível baixar um grande arquivo que inclui dados genéticos de 6326 usuários acompanhados da autodeclaração de seus fenótipos.

Um arquivo auxiliar fornece uma tabela com 673 fenótipos para cada usuário.

Dos 6326 usuários da plataforma, 1678 preencheram seus fenótipos para cor do olho.

A seguir, será construído um conjunto de dados que visa permitir a aplicação de algoritmos de aprendizado de máquina para a predição de cor do olho em indivíduos.

In [1]:
# importando bibliotecas
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

## Trabalhando com o arquivo de fenótipos

1. Leitura do arquivo contendo fenótipos de cada usuário.
2. Filtragem do usuários que declararam sua cor do olho.
3. Agrupamento das cores em 3 categorias
   * Azul, Verde, Cinza: Olhos predominantemente claros.
   * Intermediários: Castanho claro, mel, avelã (hazel).
   * Castanho.
4. Gravação do arquivo processado em um conjunto de dados auxiliar.

Obs: Nesta etapa sobraram apenas 1286 usuários, devido ao descarte de rótulos não informativos e de dados duplicados.


In [2]:
phenotypes_df = pd.read_csv("opensnp_alldata/phenotypes_202206080235.csv", sep=';')

In [3]:
phenotypes_df.head(2)

,user_id,genotype_filename,date_of_birth,chrom_sex,openhumans_name,Retrognathia (Marfan Syndrome),Eye pigmentation,Vegetarianism/Preference for Meat,Form of foot,Eye color,A+,MSG tastes...,Boldness type,Lynch Syndrome,Early Onset Heart Disease,black,Interstitial cystitis,Jewish Ancestry,Response to Enbrel,Bicuspid aortic valve,Hemochromotosis,ACT science,Hypomagnesemia,ACT math,12,Prolapsed Organ,vi/vim or Emacs,Heart Problems - Long QT Syndrome,chronically sore neck glands,Urticaria,R1b1a2a1a1b,ACT reading,Cramps,Number of Neanderthal variants,Dental decay,Miscarriage/Spontaneous Abortion,Thrombosis,Resistance To Infectious Disease,Enhanced Hippocampal Volume,Significantly increased Risk of Heart Disease,Amount of Body Hair (Male),Morton's Toe,Pressure Sensitivity of the Ear,"Dolichocephaly: Disproportionately Long, Narrow Head & Neanderthal rs12416000(A;G)",Neanderthal,Webbed toes,Thyroid Issues/Cancer,Purposefulness,Right Atrial Enlargement,mtDNA Haplogroup (PhyloTree),Lactose intolerance,"Light-skinned, European Ancestry (rs14256654)",Fainting Upon Seeing Blood/Gore/Violence,Erythromycin allergy,Crohn's Disease +rs2241880,ethnicity,Interests-General,"Increased Risk of GYN, Head & Neck Cancers (mutation on RNASEL Gene ~ Cancer Marker)",Premature Atrial Contractions,Reduced MAO-A Activity,Hidradenitis Supperativa,Scottish Ancestry,Welsh Ancestry,nosebleeds as a child,Hair and eye color Brown,Mitral Insufficiency (Regurgitation),Left Ventricular Hypertrophy,ABH Blood Group (antigens),"black skin,O+,hair Black,Eye color Brown,",Photic Sneeze Reflex (Photoptarmis),ENTP,Tricuspid Insufficiency (Regurgitation),Weight,Large tonsil crypts and tonsil stones,Uterine Fibroids,AB +,Type 2 Diabetes +rs13266634,Kell Blood Group (K/k antigens),MBTI (Myers Briggs) Type,"Throat, Stomach Cancer +rs2274223",Nicotine dependence,Thyroid Disorders +rs966423,Asparagus Metabolite Detection,Beard Color,Sexual Preferences,IQ,In both the large group of people is best for you ?for you ?,Ear - darwin's tubercle,Unable To Metabolize Common Medications (CYP3A5 non-expressor),Alzheimer's Disease,Frequency of colds/flu,Fibromyalgia,Hedonic set point,Otosclerosis,Skin color.,Ability to Tan,Little fingers(pinkies),Response to Humira,Farnsworth Munsell 100 Hue Test,FBN1 Mutation,CMV serostatus,Aquagenic pruritus (episodes),Atherosclerotic Vascular Disease +rs2943634,music ear,eczema,How do you put your glasses down?,Polycystic ovary syndrome (pcos),Impacted Canines,Primary Open Angle Glaucoma,Misophonia,Panic Disorder,Allergy to Egg Whites,Acrophobia,vitiligo,Double vision,Preterm labor,Anemic?,TMJ/TMD,ADHD? Subtype?,Response to Metformin,Autoimmune Disease,Non Alcoholic Fatty Liver,Social Level,Supraventricuar Tachycardia,"dark Blonde, hazal eyes,, 165 cm",Plantar fasciitis,Anorgasmia,maternal or mTDNA haplogroup,Supernumerary Kidney,Fish Preference,Looking at the world around you,No headaches EVER,Exercise Induced Ischemia +rs1024611,Kidd Blood Group,multiple brain aneurysms,adult a.d.d,Alcohol Consumption (per week),Alcoholism,Cigarette/Cigar Smoker,Chronic cough and single CFTR mutation,"Busy Bee, Multi tasker",Curiosity and love of research,shoe size (US MEN),Mother's eye color,Rain,Latino Ancestry,Secretor Status,hair color,impaired NSAID drug metabolism,Scoliosis,"Third molars ""wishdom teeth""",water taste,Anorexia nervosa or ednos,rs429358,immune thrombocytopenia,Spondyloarthropathy,Oxycodone/Oxycontin Effectiveness,parsonage-turner syndrome,Toenail or fingernail fungus,Arthritis,Peanut butter preference,Eye with Blue Halo,Serotonin transporter,Personality Disorder test - top result,Aphantasia,Depression,One Warrior Gene/One Non-Warrior Gene,Cocaine addiction,"Gray eyes,fair skin, dark blonde, o- blood",Craves sugar,Dermatofibroma,Essential Tremor,low appetite,PICA - eating non food stuff,introvert or extrovert,Can you smell cut-grass?,Cervical dysplasia / cancer,Critical thinking,CLL,Carrier,Medium brown skin,Online Alexithymia Questionnaire,"Medium

In [4]:
# filtrar o dataset para mostrar apenas as colunas que podem ser de interesse
colunas_interesse = ["user_id","genotype_filename", "Eye color"]
eye_color_df = phenotypes_df[colunas_interesse]

In [5]:
eye_color_df["Eye color"].value_counts()

-                                                                                                         4648
Brown                                                                                                      408
Blue                                                                                                       214
Hazel                                                                                                      124
Blue-grey                                                                                                  122
Green                                                                                                      116
Dark brown                                                                                                  88
Blue-green                                                                                                  75
Brown-green                                                                                                 68
H

In [6]:
# filtrar apenas os valores preenchidos
eye_color_df = eye_color_df[eye_color_df["Eye color"] != '-']
len(eye_color_df)

1678

In [7]:
# organizar as entradas em 3 categorias: Brown, Blue/Green/Gray, intermediate
color_map = {
    "Brown":"Brown",
    "Blue":"BGG",
    "Hazel":"Int",
    "Blue-grey":"BGG",
    "Green":"BGG",
    "Dark brown":"Brown",
    "Blue-green":"BGG",
    "Hazel (brown/green)":"Int",
    "Hazel/light brown":"Int",
    "Gray-blue":"BGG",
    "Blue-grey; broken amber collarette":"BGG",
    "hazel":"Int",
    "Dark blue":"BGG",
    "Green-hazel":"Int",
    "Green-brown":"Int",
    "brown":"Brown",
    "Green-gray":"BGG",
    "Blue-green":"BGG",
    "blue-green":"BGG",
    "blue":"BGG",
    "Blue, grey, green, changing":"BGG",
    "Blue grey":"BGG",
    "Blue with yellow parts":"BGG",
    "Blue-grey with central heterochromia":"BGG",
    "Light-mixed green":"BGG",
    "blue":"BGG",
    "blue-grey":"BGG",
    "Blue-green; amber collarette, and gray-blue ringing ":"BGG",
    "Blue with a yellow ring of flecks that make my eyes look green depending on the light or my  mood":"BGG",
    "Brown/black":"Brown",
    "Hazel (light brown, dark green, dark blue)":"Int",
    "Blue-green-grey":"BGG",
    "Green":"BGG",
    "Brown-amber":"Int",
    "blue, grey, green, changing":"BGG",
    "Amber":"Int",
    "Amber - (yellow/ocre  brown)":"Int",
    "Hazel/Light Brown":"Int",
    "Amber - (yellow/ocre  brown)":"Int"
}
    
eye_color_df["color_cat"] = eye_color_df["Eye color"].map(color_map)
eye_color_df = eye_color_df.dropna(axis=0, subset = "color_cat")
eye_color_df["color_cat"].value_counts()

BGG      698
Brown    518
Int      263
Name: color_cat, dtype: int64

In [8]:
len(eye_color_df)

1479

In [9]:
eye_color_df.head(2)

,user_id,genotype_filename,Eye color,color_cat
3,2953,2953.ftdna-illumina.1885,Dark brown,Brown
16,4135,4135.ftdna-illumina.2801,Blue-grey,BGG


In [10]:
eye_color_df_snp = eye_color_df.copy()
eye_color_df_snp.reset_index(drop = True, inplace = True)

In [11]:
eye_color_df_snp.head(2)

,user_id,genotype_filename,Eye color,color_cat
0,2953,2953.ftdna-illumina.1885,Dark brown,Brown
1,4135,4135.ftdna-illumina.2801,Blue-grey,BGG


In [12]:
eye_color_df_snp['user_id'].duplicated().sum()

193

In [13]:
eye_color_df_snp_drop_duplicates = eye_color_df_snp.drop_duplicates(subset='user_id')

In [14]:
len(eye_color_df_snp_drop_duplicates)

1286

In [15]:
# salvar estes dados
import os
os.makedirs('datasets', exist_ok=True)  
eye_color_df_snp_drop_duplicates.to_csv('datasets/eye_color_df_snp.csv', index=False)  

## Leitura dos SNPs dos arquivos individuais

Nesta etapa será construído um conjunto de dados contendo os snps que queremos analisar e o código do usuário para cada usuário.

1. O arquivo anterior possui uma coluna indicando o nome do arquivo individual com os dados genéticos de cada usuário, os valores desta coluna foram processados para criar um padrão para acessar estes arquivo individuais. (utilizando glob). Resultando em um conjunto de dados auxiliar com uma coluna contendo os nomes orginais de cada arquivo para cada suário.
2. Foi utilizado a biblioteca [snps](https://pypi.org/project/snps/) para parsear os dados genéticos de cada indivíduo em um objeto pandas dataframe.
3. O dataframe de cada indivíduo foi filtrado para conter as snps que queremos salvar e cada snp de interesse foi salva em uma lista para a criação de um dataframe contendo as snps de interesse para cada usuário.

In [16]:
# importando bibliotecas necessárias
from snps import SNPs 
import glob

### Criação de conjunto de dados auxiliar

In [17]:
def get_true_file_name():
    
    user_id_list = [] 
    doc_name_list = []
    indexes = []

    for index, row in eye_color_df_snp_drop_duplicates.iterrows():
        user_id = row['user_id']
        filename = row['genotype_filename']

        split = filename.rsplit('.')
        glob_pattern =  "user{}_file{}".format(split[0], split[2])

        try:
            doc_name = glob.glob("opensnp_alldata/{}*.txt".format(glob_pattern))[0]
            user_id_list.append(user_id)
            doc_name_list.append(doc_name)
            indexes.append(index)
        except:
            pass


    return user_id_list, doc_name_list, indexes

In [18]:
user_id_list, doc_name_list, indexes = get_true_file_name()

In [19]:
def create_dataframe(user_id_list, doc_name_list, indexes):
    return pd.DataFrame({"user_id":user_id_list, "filename":doc_name_list, "index": indexes})

In [20]:
true_filenames = create_dataframe(user_id_list, doc_name_list, indexes)

In [21]:
true_filenames_drop_duplicates = true_filenames.drop_duplicates(subset='user_id')

In [22]:
# salvar estes dados
os.makedirs('datasets', exist_ok=True)  
true_filenames_drop_duplicates.to_csv('datasets/true_filenames.csv', index=False)

In [23]:
true_filenames_drop_duplicates.head(2)

,user_id,filename,index
0,2953,opensnp_alldata\user2953_file1885_yearofbirth_...,0
1,4135,opensnp_alldata\user4135_file2801_yearofbirth_...,1


### Lista de SNPs que queremos salvar
Na célula abaixo está uma lista de SNPs com valor preditivo para cor dos olhos segundo artigos científicos da área.

In [24]:
snps_eye_color = [
    'rs12913832',
    'rs1800407',
    'rs12896399',
    'rs16891982',
    'rs1393350',
    'rs12203592',
    'rs1129038',
    'rs116363232',
    'rs1289399']

### Procedimento passando por todos os arquivos para:
* Filtrar os dados individuais
* Gerar um conjunto de dados com as SNPs da lista `snps_eye_color`

In [25]:
snp_dict = {}
for snp in snps_eye_color:
    snp_dict[snp] = []

user_id_list2 = []   
    
for index, row in true_filenames_drop_duplicates.iterrows():
    try:        
        s = SNPs(r'{}'.format(row['filename']))
        df = s.snps
        df = df[['genotype']]
        
        
        
        df_t = df.filter(items = snps_eye_color, axis=0).T #dataframe de um linha com cada snp e o conteúdo no valor

        for snp in snps_eye_color:
            try:
                snp_dict[snp].append(df_t[snp][0])
            except:
                snp_dict[snp].append('missing')
                
        user_id_list2.append(row['user_id'])
                
                
        #print(row['user_id'])
        print(len(df.filter(items = snps_eye_color, axis=0)))
        #print('-'*30)
        
    except:
        print(r"ERRO!!! NO ARQUIVO {}".format(row['filename']))
        
        
        
        

2
2
6


C:\ProgramData\Miniconda3\lib\site-packages\snps\io\reader.py:450: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


8
6


C:\ProgramData\Miniconda3\lib\site-packages\snps\io\reader.py:450: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


8


...

8


C:\ProgramData\Miniconda3\lib\site-packages\snps\io\reader.py:450: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


9
6
7
9


C:\ProgramData\Miniconda3\lib\site-packages\snps\io\reader.py:450: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


9


In [26]:
# Criando o conjunto de dados utilizando o dicionário e as listas
snp_dict_user = snp_dict.copy()
snp_dict_user['user_id'] = user_id_list2
df_final = pd.DataFrame(snp_dict_user)

In [27]:
df_final.head(2)

,rs12913832,rs1800407,rs12896399,rs16891982,rs1393350,rs12203592,rs1129038,rs116363232,rs1289399,user_id
0,missing,CC,missing,missing,missing,missing,TC,missing,missing,2953
1,missing,TC,missing,missing,missing,missing,TT,missing,missing,4135


In [28]:
df_final_drop_duplicates = df_final.drop_duplicates()

In [29]:
len(df_final_drop_duplicates)

1264

In [30]:
# Salvando o dataframe
df_final_drop_duplicates.to_csv('datasets/users_snps.csv', index=False)